<a href="https://colab.research.google.com/github/sarahaptumich/capstone/blob/shaptonstall/brace.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:

#Mount google drive
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [17]:
%cd /content/drive/MyDrive/Github/capstone/brace

/content/drive/MyDrive/Github/capstone/brace


In [18]:
%ls

annotations/  dataset_pytorch.py  __pycache__/  utils/
dataset/      figures/            README.md     videos_info.csv


In [20]:
import sys
sys.path.append('/content/drive/MyDrive/Github/capstone/brace')

In [109]:
import pandas as pd
import numpy as np
import torch
from torch.utils.data import DataLoader
from torch import nn, optim
import dataset_pytorch
from pathlib import Path
from torch.utils.data import DataLoader, TensorDataset

In [42]:
sequences_path = Path('/content/drive/MyDrive/Github/capstone/brace/dataset')  # Adjust this path to where your dataset is located
annotations_path = Path('/content/drive/MyDrive/Github/capstone/brace/annotations')
df_ = pd.read_csv(Path('/content/drive/MyDrive/Github/capstone/brace/annotations/sequences.csv'))

In [43]:
# train_df = pd.read_csv('/content/drive/MyDrive/Github/capstone/brace/annotations/sequences_train.csv')
# test_df = pd.read_csv('/content/drive/MyDrive/Github/capstone/brace/annotations/sequences_test.csv')

In [64]:
segments= pd.read_csv('/content/drive/MyDrive/Github/capstone/brace/annotations/segments.csv')
segments

,video_id,seq_idx,start_frame,end_frame,dance_type,dance_type_id,dancer,dancer_id,year,uid
0,3rIk56dcBTM,0,851,1221,toprock,0,el niño,10,2011,3rIk56dcBTM.851.1221
1,3rIk56dcBTM,0,1234,1330,powermove,1,el niño,10,2011,3rIk56dcBTM.1234.1330
2,3rIk56dcBTM,0,1348,1555,footwork,2,el niño,10,2011,3rIk56dcBTM.1348.1555
3,3rIk56dcBTM,0,1571,1645,powermove,1,el niño,10,2011,3rIk56dcBTM.1571.1645
4,3rIk56dcBTM,1,1751,2085,toprock,0,roxrite,47,2011,3rIk56dcBTM.1751.2085
...,...,...,...,...,...,...,...,...,...,...
1347,zvWiTWT3T2M,5,4198,4270,toprock,0,alkolil,0,2020,zvWiTWT3T2M.4198.4270
1348,zvWiTWT3T2M,5,4281,4391,powermove,1,alkolil,0,2020,zvWiTWT3T2M.4281.4391
1349,zvWiTWT3T2M,5,4407,4563,footwork,2,alkolil,0,2020,zvWiTWT3T2M.4407.4563
1350,zvWiTWT3T2M,5,4576,4654,powermove,1,alkolil,0,2020,zvWiTWT3T2M.4576.4654


In [51]:
train_df = pd.read_csv(annotations_path/'sequences_train.csv')
train_df = df_[df_.uid.isin(train_df.uid)]
test_df = pd.read_csv(annotations_path/'sequences_test.csv')
test_df = df_[df_.uid.isin(test_df.uid)]

In [108]:
# # Initialize the datasets and saves the into a pytorch and a tensor flow
# brace_train = dataset_pytorch.BraceDataset(sequences_path, train_df)
# brace_test = dataset_pytorch.BraceDataset(sequences_path, test_df)
# y_train = brace_train.clip_labels

# y_test = brace_test.clip_labels

# np.save('y_train_np.npy', y_train)  # Saves the array to a file
# np.save('y_test_np.npy', y_test)  # Saves the array to a file


# x_train = np.array([brace_train[i][0] for i in range(len(brace_train))])
# x_train= torch.tensor(x_train)
# torch.save(x_train, 'x_train_tensor.pt')
# x_test = np.array([brace_test[i][0] for i in range(len(brace_test))])
# x_train= torch.tensor(x_test)
# torch.save(x_test, 'x_test_tensor.pt')

In [107]:
# To load data
y_train = np.load('x_train_np.npy')
y_test = np.load('x_test_np.npy')
x_train = torch.load('x_train_tensor.pt')
x_test = torch.load('x_test_tensor.pt')

In [113]:
x_train.shape, y_train.shape

(torch.Size([146, 900, 17, 2]), (929,))

In [110]:
train_data = TensorDataset(x_train, y_train)
dataloader = DataLoader(train_data, batch_size=64, shuffle=True)

TypeError: 'int' object is not callable

In [79]:
class SimpleLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(SimpleLSTM, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        _, (h_n, _) = self.lstm(x)
        x = self.fc(h_n.squeeze(0))
        return x

In [83]:
input_size = 2  # Assuming each keypoint is (x, y)
hidden_size = 100  # Example size
num_classes = 3  # Adjust based on your dataset

model = SimpleLSTM(input_size, hidden_size, num_classes)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [85]:
num_epochs = 10
for epoch in range(num_epochs):
    for sequences, labels in brace_train:
        sequences = torch.tensor(sequences, dtype=torch.float)
        labels = torch.tensor(y_train, dtype=torch.long)

        # Forward pass
        outputs = model(sequences)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

ValueError: Expected input batch_size (900) to match target batch_size (929).